In [20]:
import gensim
from collections import Counter
from itertools import combinations
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from math import log

In [2]:
#data preparation
data_df = pd.read_csv('order_products__train.csv')

data_dict = {}

for index,row in data_df.iterrows():
    if row['order_id'] not in data_dict.keys():
        data_dict[row['order_id']] = [str(row['product_id'])]
    else:
        data_dict[row['order_id']].append(str(row['product_id']))

    

In [3]:
data_list = [v for _,v in data_dict.items()]

In [8]:
unigrams_cnt = Counter()
bigrams_cnt = Counter()
for text in data_list:
    for x in text:
        unigrams_cnt[x] += 1
    for x, y in map(sorted, combinations(text, 2)):
        bigrams_cnt[(x, y)] += 1


In [9]:
id2uni = {}
uni2id = {}
it = 0

for uni,_ in unigrams_cnt.items():
    id2uni[it] = uni
    uni2id[uni] = it
    it +=1    

In [14]:
sum_uni = sum(unigrams_cnt.values())
sum_bi = sum(bigrams_cnt.values())

In [21]:
pmi_samples = Counter()
data, rows, cols = [], [], []
for (x, y), n in bigrams_cnt.items():
    rows.append(uni2id[x])
    cols.append(uni2id[y])
    data.append(log((n / sum_bi) / (unigrams_cnt[x] / sum_uni) / (unigrams_cnt[y] / sum_uni)))
    pmi_samples[(x, y)] = data[-1]
PMI = csc_matrix((data, (rows, cols)))

In [23]:
U,_,_ = svds(PMI, k = 300)
norms = np.sqrt(np.sum(np.square(U), axis=1, keepdims=True))
U /= np.maximum(norms, 1e-7)

In [27]:
k = 10
x = '19204'
dd = np.dot(U, U[uni2id[x]])
s = ''

for i in np.argpartition(-1 * dd, k + 1)[:k + 1]:
    if id2uni[i] == x: continue
    s += '(%s, %.3lf) ' % (id2uni[i], dd[i])
print('%s, %d\n %s' % (x, unigrams_cnt[x], s))
# 

19204, 74
 (22190, 0.469) (10968, 0.443) (21527, 0.382) (37599, 0.433) (25587, 0.381) (1577, 0.369) (18406, 0.353) (1306, 0.311) (23263, 0.332) (39248, 0.310) 
